In [3]:
import os
import zipfile
import pandas as pd

''' zip_dir 내의 모든 zip 파일을 풀어, extract_to 경로에 압축 해제 '''
def extract_zip(zip_dir, extract_to):
    for file in os.listdir(zip_dir):
        if file.endswith('.zip'):
            # 압축 해제해야 하는 파일 주소 (파일명 포함)
            zip_path = os.path.join(zip_dir, file)
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)


''' data_dir 내의 모든 tsv 파일을 불러와 하나의 데이터프레임으로 합침 '''
def load_tsv_files(extract_to):
    all_data = []
    for root, dirs, files in os.walk(extract_to):
        for file in files:
            if file.endswith('.tsv'):
                file_path = os.path.join(root, file)
                try:
                    # 잘못된 줄 건너띄기
                    df = pd.read_csv(file_path, sep='\t', on_bad_lines='skip')
                    all_data.append(df)
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
    return pd.concat(all_data, ignore_index=True)

In [4]:
''' 압축 해제할 디렉토리와 해제될 경로 설정 '''
# Training
zip_dir_training = ("C:\KUBIG contest\AIhub_rawdata\Training_zip").replace('\\', '/')
extract_to_training = ("C:\KUBIG contest\AIhub_rawdata\Training_unzip").replace('\\', '/')
# Validation
zip_dir_validation = ("C:\KUBIG contest\AIhub_rawdata\Validation_zip").replace('\\', '/')
extract_to_validation = ("C:\KUBIG contest\AIhub_rawdata\Validation_unzip").replace('\\', '/')

''' 압축 파일 풀기 '''
extract_zip(zip_dir_training, extract_to_training)
print('---------------Training 압축 해제 완료!-------------------')
extract_zip(zip_dir_validation, extract_to_validation)
print('---------------Validation 압축 해제 완료!-------------------')

''' TSV 파일 불러와서 하나의 데이터프레임으로 합치기 '''
train_dataset = load_tsv_files(extract_to_training)  # (378562, 7)
valid_dataset = load_tsv_files(extract_to_validation)  # (46804, 7)

---------------Training 압축 해제 완료!-------------------
---------------Validation 압축 해제 완료!-------------------


# single-turn 데이터 만들기

In [5]:
''' id와 utterance_id를 기준으로 정렬 '''
train_dataset = train_dataset.sort_values(by=['id', 'utterance_id']).reset_index(drop=True)
valid_dataset = valid_dataset.sort_values(by=['id', 'utterance_id']).reset_index(drop=True)

''' 대화를 input, output 쌍으로 분리 '''
def create_input_output_pairs(df):
    pairs = []
    current_input = None
    for _, row in df.iterrows():
        if row['utterance_type'] == 0:  # 감정화자
            current_input = row['utterance_text']
        elif row['utterance_type'] == 1 and current_input is not None:  # 공감화자
            pairs.append((current_input, row['utterance_text']))
            current_input = None
    return pairs

train_pairs = create_input_output_pairs(train_dataset)
valid_pairs = create_input_output_pairs(valid_dataset)

''' 데이터프레임으로 변환 '''
train_df_single = pd.DataFrame(train_pairs, columns=['input', 'output'])
train_df_single['type'] = 'single'
valid_df_single = pd.DataFrame(valid_pairs, columns=['input', 'output'])
valid_df_single['type'] = 'single'

# multi-turn 데이터 만들기

In [6]:
def create_multi_turn(df):
    multi_turn_data = []

    # id별로 대화를 그룹화
    grouped = df.groupby('id')

    # 각 그룹에 대해 멀티턴 형식으로 데이터를 생성
    for group_id, group in grouped:
        turns = group.to_dict(orient='records')
        for i in range(0, len(turns) - 2, 2):  # 2-step window
            if i + 2 < len(turns):
                if turns[i]['utterance_type'] == 0 and turns[i+1]['utterance_type'] == 1 and turns[i+2]['utterance_type'] == 0:
                    input = (
                        f"{turns[i]['utterance_text']}</s>{turns[i+1]['utterance_text']}</s>{turns[i+2]['utterance_text']}"
                    )
                    output = f"{turns[i+3]['utterance_text']}" if i + 3 < len(turns) and turns[i+3]['utterance_type'] == 1 else None
                    
                    # output이 존재하는 경우에만 데이터 추가
                    if output is not None:
                        multi_turn_data.append({
                            'input': input,
                            'output': output
                        })
    return pd.DataFrame(multi_turn_data)

train_df_multiturn = create_multi_turn(train_dataset)
train_df_multiturn['type'] = 'multi'
valid_df_multiturn = create_multi_turn(valid_dataset)
valid_df_multiturn['type'] = 'multi'

# single-turn, multi-turn 합치기

In [14]:
train_df = pd.concat([train_df_single, train_df_multiturn], ignore_index=True)  # (338148, 3)
valid_df = pd.concat([valid_df_single, valid_df_multiturn], ignore_index=True)  # (41926, 3)

train_df.to_csv('C:/KUBIG contest/AIhub_data/train_df.csv', index=False)
valid_df.to_csv('C:/KUBIG contest/AIhub_data/valid_df.csv', index=False)

print('train_df >>')
display(train_df.head())
print('valid_df >>')
display(valid_df.head())

train_df >>


,input,output,type
0,"과장님, 빨리 퇴근하고 싶어요. 야근 작업이 끝이 없어서 너무 화가 나요.",아직도 야근 중인가요?,single
1,팀장님이 마치기 전에 5분 전에 내일 아침 미팅 자료 좀 검토해 놓으라고 하셨거든요.,"어제부터 몸살기가 있어서 고생하더니, 일찍 퇴근도 못하고 고단해서 어째요.",single
2,그러니까요. 퇴근하자마자 감기약을 먹고 쉬려고 했는데 머리에서 열이 솟구치는 기분이에요.,"에구, 하필 아픈 날 어째요. 그것도 프로젝트 마치고 몸살이 난 건데요. 직장인의 ...",single
3,"그것뿐이 아니에요, 내일 검색해도 될 것을 지금 하라고 하셨어요.",혹시 몸이 아프다고 말씀드려보면 어떨까요?,single
4,핑계라고 생각하실 것 같아요. 하루 종일 아픈 내색을 안 하려고 애썼거든요.,감정화자씨 워낙 일에 대해서 철저하니까 내색을 안 했겠지요. 감정화자씨 보면 진정한...,single


valid_df >>


,input,output,type
0,지난 달부터 엄마가 다시 일하시기 시작했어. 활력을 찾은 엄마 모습에 기분이 너무 ...,"와, 정말 잘 됐다! 너 뿌듯하겠다. 나도 이렇게 기쁜걸!",single
1,엄마가 이렇게 자신감을 가지고 사회생활을 다시 하신다는 게 믿기지가 않아. 그만큼 ...,"그럼, 너 스트레스 때문에 고생 엄청 했잖아. 기다리던 엄마 모습을 드디어 봤으니,...",single
2,어 엄마 항암 치료받으시던 지난 2년간 다시 일어나실 수 있을까 걱정 정말 많았어....,"나도 알지. 엄마 생각 항상 먼저 하면서, 보살피던 너의 지극 정성을 어떻게 잊을 ...",single
3,"당연히 해야 할 일이었지. 공감화자가 그렇게 말해주니, 그때 했던 고생을 칭찬받는 ...",와 놀랍다. 네가 어머님 의지력을 닮았구나! 아직도 몸이 힘드실 텐데 말이야. 나라...,single
4,"역시 공감화자 넌 다정해! 나는 그런 애교스러운 모습은 없는데, 오늘 처음으로 엄마...","감성화자야 너무 잘했다! 맞아, 부모님들은 그런 고마움이나 사랑의 표현을 기다리고 ...",single
